In [1]:
import datetime
import fileinput
import glob
import json
import os
import re
import subprocess
import sys

## Data

In [2]:
def read_mut_data(path): 
    input_file = open(path, "r")
    project_targets = json.loads(input_file.read())
    input_file.close()
    return project_targets

## Test runner

In [3]:
def install_parent_module(target):
    os.chdir(target["parent-module-path"])
    process = subprocess.Popen(['mvn', 'clean', 'install', '-DskipTests'],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked a clean install of the parent module with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))
    output_to_log("---------------------------------------------------")
    os.chdir(experiment_directory)
    # if compilation fails
    if ("COMPILATION ERROR" in stdout.decode("utf-8")):
        output_to_log("[LOG] Skipping this mutant, restoring original " + target["parent-file"])
        return False
    return True

In [4]:
def run_mvn_test(test, mode):
    result = ""
    if (mode == "original"):
        output_to_log("[LOG] Running original test " + test["original-test"])
        os.chdir(test["original-test-module-path"])
        output_to_log("[LOG] In test module directory " + os.getcwd())
        # run original test
        run_test_method = "-Dtest=" + test["original-test"]
        process = subprocess.Popen(['mvn', 'test', run_test_method],
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.PIPE)
        output_to_log("[LOG] Invoked tests with {}".format(process.args))
        stdout, stderr = process.communicate()
        result = stdout.decode("utf-8")
        output_to_log(result)
        output_to_log(stderr.decode("utf-8"))
    else:
        output_to_log("[LOG] Running parameterized test " + test["param-test-class"])
        os.chdir(test["param-test-module-path"])
        output_to_log("[LOG] In test module directory " + os.getcwd())
        # run parameterized test
        run_test_class = "-Dtest=" + test["param-test-class"]
        process = subprocess.Popen(['mvn', 'test', run_test_class],
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.PIPE)
        output_to_log("[LOG] Invoked tests with {}".format(process.args))
        stdout, stderr = process.communicate()
        result = stdout.decode("utf-8")
        output_to_log(result)
        output_to_log(stderr.decode("utf-8"))
    
    output_to_log("---------------------------------------------------")

    # store result here
    result = re.search(r"(Tests run:.*)\n", result).group(1)

    # cd back into experiment directory
    os.chdir(experiment_directory)
    output_to_log("[LOG] In experiment directory " + os.getcwd())
    output_to_log("---------------------------------------------------")
    
    return result

## Util

In [5]:
def output_to_log(message):
    with open(mut_mutation_log_path, 'a+') as logfile:
        logfile.write(message)
        logfile.write("\n")

In [6]:
def cleanup_existing_results():
    output_to_log("[LOG] Removing pre-existing mutants from current working directory")
    os.system('rm -rf *.java/')
    output_to_log("[LOG] Cleaning up LittleDarwin results")
    os.system('rm -rf ' + littledarwin_results_path)

In [7]:
def get_mutant_files(parent):
    mutants = ('./' + parent + '/[0-9]*.java')
    mutant_files = sorted(glob.glob(mutants))
    return mutant_files

In [8]:
def copy_littledarwin_results(parent):
    output_to_log("[LOG] Copying mutants of " + parent)
    os.system('cp -r ' + littledarwin_results_path + parent + " ./")

In [9]:
def run_littledarwin(path):
    output_to_log("[LOG] Running littledarwin on " + path)
    process = subprocess.Popen(['python3', '-m', 'littledarwin', '-m', '-v', '-p', path, '--all'],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked LitteDarwin with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))

In [10]:
def find_if_mutant_is_in_mut(target, mutant_file):
    search_string = "line number in original file: "
    found = ""
    with open(mutant_file, 'r') as fp:
        for l_no, line in enumerate(fp):
            # search string
            if search_string in line:
                found = line
                print("Processing mutant file", mutant_file)
                print("mutant found:", found)
                break
                
    result = int(re.search(r"(line number in original file:\s)(\d+)", found).group(2))
    return result >= target["mut-start-line"] and result <= target["mut-end-line"]

In [11]:
def copy_mutant_files_to_output_directory(target):
    mutants_in_experiment_directory = experiment_directory + target["parent-file"]
    path_for_mutants = output_directory + target["mut"] + "/"
    output_to_log("[LOG] Copying littledarwin mutants to " + path_for_mutants)
    process = subprocess.Popen(['cp', '-r', mutants_in_experiment_directory, path_for_mutants],
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE)
    output_to_log("[LOG] Invoked copying with {}".format(process.args))
    stdout, stderr = process.communicate()
    output_to_log(stdout.decode("utf-8"))
    output_to_log(stderr.decode("utf-8"))

## Mutation analysis

In [12]:
def run_experiment_with_mutant_file(target, mutant_file, original_file):
    output_to_log("[LOG] Replacing original " + target["parent-file"] + " with " + mutant_file)
    replace_original_with_mutant = "cp " + mutant_file + " " + target["parent-path"] + target["parent-file"]
    output_to_log("[LOG] Invoking " + replace_original_with_mutant)
    os.system(replace_original_with_mutant)
    
    # cd into parent module, make a clean install, cd back into experiments directory
    mutant_compilable = install_parent_module(target)
    
    if mutant_compilable:
        target_report = {
            "mut": target["mut"],
            "CUT": target["parent-fqn"],
            "mutant-file": mutant_file,
            "mutant-within-mut": True,
            "tests": []
        }

        for t in range(len(target["tests"])):
            test = {}
            test["test"] = target["tests"][t]["original-test"]
            # execute original test
            test["result_original"] = run_mvn_test(target["tests"][t], "original")
            # execute parameterized test
            test["result-parameterized"] = run_mvn_test(target["tests"][t], "parameterized")
            target_report["tests"].append(test)

        mutation_report_file = output_directory + target["mut"] + "/mutation-report.json"

        with open(mutation_report_file, 'a+') as outfile:
            json.dump(target_report, outfile, indent=4)
            outfile.write(",\n")
        
    output_to_log("[LOG] Resetting " + target["parent-file"] + " to original")
    replace_mutant_with_original = "cp " + original_file + " " + target["parent-path"] + target["parent-file"]
    output_to_log("[LOG] Invoking " + replace_mutant_with_original)
    os.system(replace_mutant_with_original)

In [13]:
def run_mutation_analysis(target):
    global mut_mutation_log_path
    mut_mutation_log_path = output_directory + target["mut"] + "/mutation-analysis.log"
    os.makedirs(os.path.dirname(mut_mutation_log_path), exist_ok=True)
    
    output_to_log("[LOG] Initiating analysis at " + str(datetime.datetime.now()))
    output_to_log("---------------------------------------------------")
    
    # housekeeping
    cleanup_existing_results()
    output_to_log("---------------------------------------------------")

    output_to_log("[LOG] Working with " + target["mut"])
    output_to_log("[LOG] Generating mutants for its parent " + target["parent-file"])
    output_to_log("---------------------------------------------------")
    
    # run littledarwin
    run_littledarwin(target["parent-path"])    
    output_to_log("---------------------------------------------------")
    
    # copy littledarwin results to current directory
    copy_littledarwin_results(target["parent-file"])
    output_to_log("---------------------------------------------------")

    # get list of mutant files
    mutant_files = get_mutant_files(target["parent-file"])
    output_to_log("[LOG] Generated " + str(len(mutant_files)) + " mutants")
    output_to_log("---------------------------------------------------")

    # get original file
    original_file = glob.glob(r'./' + target["parent-file"] + '/original.java')[0]
    
    for f in range(len(mutant_files)):
        if find_if_mutant_is_in_mut(target, mutant_files[f]):
            run_experiment_with_mutant_file(target, mutant_files[f], original_file)
            output_to_log("---------------------------------------------------")
    
    # copy all mutants to output folder
    copy_mutant_files_to_output_directory(target)
    
    output_to_log("---------------------------------------------------")
    output_to_log("[LOG] Finishing analysis at " + str(datetime.datetime.now()))
    
    output_to_log("====== END OF THE LINE ======")

In [14]:
# for each mut:
## invoke LitteDarwin to generate mutants
## for each mutant:
### replace original parent class with mutated
### run original test > save log
### run parameterized test > save log
### restore original parent class

def main():
    global experiment_directory
    global output_directory
    global littledarwin_results_path
    global oo_string
    global po_string
    global co_string

    experiment_directory = "/home/dee/dev/fakir/proze-experiments/mutation-analysis/"
    output_directory = experiment_directory + "output/"
    littledarwin_results_path = "/home/dee/anaconda3/lib/python3.8/site-packages/littledarwin/LittleDarwinResults/"
    
    os.chdir(experiment_directory)
    targets_pdfbox = read_mut_data("/home/dee/dev/fakir/proze-experiments/mutation-analysis/pdfbox/muts-pdfbox.json")

    print("Found", len(targets_pdfbox), "MUTs for PDFBox")    
    
    print("Working with PDFBox MUTs")
    for target in targets_pdfbox:
        print("Currently processing", target["mut"])
        print("Details:", target)
        run_mutation_analysis(target)

In [15]:
main()

Found 2 MUTs for PDFBox
Working with PDFBox MUTs
Currently processing PDDocument_save_java_lang_String
Details: {'mut': 'PDDocument_save_java_lang_String', 'mut-start-line': 1324, 'mut-end-line': 1327, 'parent-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/', 'parent-file': 'PDDocument.java', 'parent-fqn': 'org.apache.pdfbox.pdmodel.PDDocument', 'parent-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/src/main/java/org/apache/pdfbox/pdmodel/', 'tests': [{'original-test-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/', 'original-test': 'COSArrayListTest#removeIndirectObject', 'param-test-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/proze-tests-mutation/', 'param-test-class': 'TestProzeGen_PDDocument_save_java_lang_String_removeIndirectObject'}, {'original-test-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/', 'original-test': 'COSArrayListTest#removeSingleDirectObject', 'param-test-module-path': '/home

Processing mutant file ./PDDocument.java/186.java
mutant found: ----> line number in original file: 1308

Processing mutant file ./PDDocument.java/187.java
mutant found: ----> line number in original file: 1307

Processing mutant file ./PDDocument.java/188.java
mutant found: ----> line number in original file: 1306

Processing mutant file ./PDDocument.java/189.java
mutant found: ----> line number in original file: 1227

Processing mutant file ./PDDocument.java/19.java
mutant found: ----> line number in original file: 883

Processing mutant file ./PDDocument.java/190.java
mutant found: ----> line number in original file: 1223

Processing mutant file ./PDDocument.java/191.java
mutant found: ----> line number in original file: 1104

Processing mutant file ./PDDocument.java/192.java
mutant found: ----> line number in original file: 1101

Processing mutant file ./PDDocument.java/193.java
mutant found: ----> line number in original file: 1085

Processing mutant file ./PDDocument.java/194.jav

Processing mutant file ./PDDocument.java/297.java
mutant found: ----> line number in original file: 1305

Processing mutant file ./PDDocument.java/298.java
mutant found: ----> line number in original file: 1285

Processing mutant file ./PDDocument.java/299.java
mutant found: ----> line number in original file: 1266

Processing mutant file ./PDDocument.java/3.java
mutant found: ----> line number in original file: 1749

Processing mutant file ./PDDocument.java/30.java
mutant found: ----> line number in original file: 504

Processing mutant file ./PDDocument.java/300.java
mutant found: ----> line number in original file: 1249

Processing mutant file ./PDDocument.java/301.java
mutant found: ----> line number in original file: 1222

Processing mutant file ./PDDocument.java/302.java
mutant found: ----> line number in original file: 1201

Processing mutant file ./PDDocument.java/303.java
mutant found: ----> line number in original file: 1182

Processing mutant file ./PDDocument.java/304.java


Currently processing PDFont_getStringWidth_java_lang_String
Details: {'mut': 'PDFont_getStringWidth_java_lang_String', 'mut-start-line': 362, 'mut-end-line': 375, 'parent-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/', 'parent-file': 'PDFont.java', 'parent-fqn': 'org.apache.pdfbox.pdmodel.font.PDFont', 'parent-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/src/main/java/org/apache/pdfbox/pdmodel/font/', 'tests': [{'original-test-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/pdfbox/', 'original-test': 'PDFontTest#testSoftHyphen', 'param-test-module-path': '/home/dee/dev/fakir/proze-experiments/pdfbox/proze-tests-mutation/', 'param-test-class': 'TestProzeGen_PDFont_getStringWidth_java_lang_String_testSoftHyphen'}]}
Processing mutant file ./PDFont.java/1.java
mutant found: ----> line number in original file: 613

Processing mutant file ./PDFont.java/10.java
mutant found: ----> line number in original file: 154

Processing mutant file ./PDFo

Processing mutant file ./PDFont.java/38.java
mutant found: ----> line number in original file: 363

Processing mutant file ./PDFont.java/39.java
mutant found: ----> line number in original file: 325

Processing mutant file ./PDFont.java/4.java
mutant found: ----> line number in original file: 456

Processing mutant file ./PDFont.java/40.java
mutant found: ----> line number in original file: 325

Processing mutant file ./PDFont.java/41.java
mutant found: ----> line number in original file: 253

Processing mutant file ./PDFont.java/42.java
mutant found: ----> line number in original file: 253

Processing mutant file ./PDFont.java/43.java
mutant found: ----> line number in original file: 247

Processing mutant file ./PDFont.java/44.java
mutant found: ----> line number in original file: 235

Processing mutant file ./PDFont.java/45.java
mutant found: ----> line number in original file: 229

Processing mutant file ./PDFont.java/46.java
mutant found: ----> line number in original file: 200

P